In [69]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import grangercausalitytests, adfuller
from tqdm import tqdm_notebook
from itertools import product
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [70]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
from datetime import datetime

filepath = '/content/drive/MyDrive/MOISTURE_MINDS/combined_data.csv'
custom_date_parser = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S")
df = pd.read_csv(filepath ,
                 parse_dates=['ttime'],
                date_parser=custom_date_parser)


In [72]:
swap_list = ['pm1', 'pm2', 'pm3', 'am', 'lum', 'st','pres','humd','temp','sm']

# Swapping the columns
df = df.reindex(columns=swap_list)
print(df.head())

    pm1  pm2  pm3   am      lum     st      pres   humd   temp      sm
0  0.83  0.0  0.0  0.0   6547.0  25.65  92849.25  90.57  23.02  7463.0
1  3.63  0.0  0.0  3.2  15001.0  25.65  92849.25  90.57  23.02  7813.0
2  1.39  0.0  0.0  0.0  10249.0  25.65  92848.31  88.37  23.47  7576.0
3  4.19  0.0  0.0  4.8  13863.0  25.66  92848.31  88.37  23.47  7692.0
4  1.95  0.0  0.0  0.0  12636.0  25.66  92844.82  85.26  23.64  7576.0


In [ ]:
# from matplotlib import pyplot as plt
# x = df['ttime'].values
# print(type(x))
# y = df['sm'].values
# plt.plot(x,y)
# plt.show()

In [73]:
cols = list(df)[0:]
#Date and volume columns are not used in training. 
print(cols)

['pm1', 'pm2', 'pm3', 'am', 'lum', 'st', 'pres', 'humd', 'temp', 'sm']


In [74]:
df_for_training = df[cols].astype(float)

In [75]:
print("before training")
df_for_training

before training


,pm1,pm2,pm3,am,lum,st,pres,humd,temp,sm
0,0.83,0.00,0.00,0.0,6547.0,25.65,92849.25,90.57,23.02,7463.0
1,3.63,0.00,0.00,3.2,15001.0,25.65,92849.25,90.57,23.02,7813.0
2,1.39,0.00,0.00,0.0,10249.0,25.65,92848.31,88.37,23.47,7576.0
3,4.19,0.00,0.00,4.8,13863.0,25.66,92848.31,88.37,23.47,7692.0
4,1.95,0.00,0.00,0.0,12636.0,25.66,92844.82,85.26,23.64,7576.0
...,...,...,...,...,...,...,...,...,...,...
39502,1.67,1.11,27.38,4.8,7661.0,29.24,93571.75,24.39,30.07,327.0
39503,0.55,1.11,28.49,1.6,384.0,29.17,93571.75,24.39,30.07,291.0
39504,0.55,1.67,27.93,4.0,8386.0,29.17,93558.36,22.99,31.55,327.0
39505,1.11,1.67,27.93,2.4,8954.0,29.17,93541.99,20.85,32.68,328.0


In [76]:
scaler = StandardScaler()
df_for_training_scaled = scaler.fit_transform(df_for_training)

In [77]:
df_for_training_scaled

array([[-0.73004295, -2.01902897, -2.11504421, ...,  0.4854903 ,
         0.12151653,  1.05193958],
       [ 1.93189622, -2.01902897, -2.11504421, ...,  0.4854903 ,
         0.12151653,  1.16887899],
       [-0.19765511, -2.01902897, -2.11504421, ...,  0.39240916,
         0.2196956 ,  1.0896943 ],
       ...,
       [-0.99623686, -0.72568933, -1.12670559, ..., -2.37379311,
         1.9825553 , -1.33228799],
       [-0.46384903, -0.72568933, -1.12670559, ..., -2.46433568,
         2.22909385, -1.33195387],
       [-0.99623686, -1.15938406, -1.10688928, ..., -2.46433568,
         2.22909385, -1.34398193]])

In [78]:
train_size = int(len(df)*0.75)
test_size = len(df) - train_size
df_train = df_for_training_scaled[:train_size]
df_test = df_for_training_scaled[train_size:]

In [79]:
df_test

array([[-0.46384903, -0.72568933, -0.75090337, ...,  0.88446992,
        -1.20281001, -1.26412901],
       [ 0.0685388 , -0.72568933, -0.75090337, ...,  0.88446992,
        -1.2944438 , -1.28050053],
       [ 0.0685388 , -0.72568933, -0.75090337, ...,  0.88446992,
        -1.2944438 , -1.26412901],
       ...,
       [-0.99623686, -0.72568933, -1.12670559, ..., -2.37379311,
         1.9825553 , -1.33228799],
       [-0.46384903, -0.72568933, -1.12670559, ..., -2.46433568,
         2.22909385, -1.33195387],
       [-0.99623686, -1.15938406, -1.10688928, ..., -2.46433568,
         2.22909385, -1.34398193]])

In [80]:
trainX = []
trainY = []

n_future = 1   # Number of days we want to look into the future based on the past days.
n_past = 14  # Number of past days we want to use to predict the future.

#Reformat input data into a shape: (n_samples x timesteps x n_features)
#In my example, my df_for_training_scaled has a shape (12823, 5)
#12823 refers to the number of data points and 5 refers to the columns (multi-variables).
for i in range(n_past, len(df_train) - n_future +1):
    trainX.append(df_train[i - n_past:i, 0:df_for_training.shape[1]]-1)
    trainY.append(df_train[i + n_future - 1:i + n_future, -1])

trainX, trainY = np.array(trainX), np.array(trainY)

print('trainX shape == {}.'.format(trainX.shape))
print('trainY shape == {}.'.format(trainY.shape))

trainX shape == (29616, 14, 10).
trainY shape == (29616, 1).


In [81]:
df_test

array([[-0.46384903, -0.72568933, -0.75090337, ...,  0.88446992,
        -1.20281001, -1.26412901],
       [ 0.0685388 , -0.72568933, -0.75090337, ...,  0.88446992,
        -1.2944438 , -1.28050053],
       [ 0.0685388 , -0.72568933, -0.75090337, ...,  0.88446992,
        -1.2944438 , -1.26412901],
       ...,
       [-0.99623686, -0.72568933, -1.12670559, ..., -2.37379311,
         1.9825553 , -1.33228799],
       [-0.46384903, -0.72568933, -1.12670559, ..., -2.46433568,
         2.22909385, -1.33195387],
       [-0.99623686, -1.15938406, -1.10688928, ..., -2.46433568,
         2.22909385, -1.34398193]])

In [82]:
testX = []
testY = []

n_future = 1   # Number of days we want to look into the future based on the past days.
n_past = 14  # Number of past days we want to use to predict the future.

#Reformat input data into a shape: (n_samples x timesteps x n_features)
#In my example, my df_for_training_scaled has a shape (12823, 5)
#12823 refers to the number of data points and 5 refers to the columns (multi-variables).
for i in range(n_past, len(df_test) - n_future +1):
    testX.append(df_test[i - n_past:i, 0:df_test.shape[1]]-1)
    testY.append(df_test[i + n_future - 1:i + n_future, -1])

# trainX, trainY = np.array(trainX), np.array(trainY)
testX, testY = np.array(testX),np.array(testY)

In [83]:
testY

array([[-1.26479724],
       [-1.28116876],
       [-1.26446313],
       ...,
       [-1.33228799],
       [-1.33195387],
       [-1.34398193]])

In [84]:
z = np.repeat(testY, df_for_training.shape[1], axis=-1)
z = scaler.inverse_transform(z)[:,-1]
z

array([529., 480., 530., ..., 327., 328., 292.])

In [ ]:
# print((df_for_training[i + n_future - 1:i + n_future, 9]))

In [ ]:
# model = Sequential()
# model.add(LSTM(64, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
# model.add(LSTM(32, activation='relu', return_sequences=False))
# model.add(Dropout(0.2))
# model.add(Dense(trainY.shape[1]))

# model.compile(optimizer='adam', loss='mse')
# model.summary()

In [85]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(128, input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
model.add(tf.keras.layers.LeakyReLU(alpha=0.5)) 
model.add(tf.keras.layers.LSTM(128, return_sequences=True))
model.add(tf.keras.layers.LeakyReLU(alpha=0.5)) 
model.add(tf.keras.layers.Dropout(0.3)) 
model.add(tf.keras.layers.LSTM(64, return_sequences=False))
model.add(tf.keras.layers.Dropout(0.3)) 
model.add(tf.keras.layers.Dense(1))

In [93]:
from keras.callbacks import ModelCheckpoint
# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
#                                                     patience=2,
#                                                     mode='min')

checkpoint = ModelCheckpoint("y.hdf5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto', period=1)

model.compile(loss=tf.losses.MeanSquaredError(),
              optimizer=tf.optimizers.Adam(),
              metrics=[tf.metrics.MeanAbsoluteError()])

# history = model.fit_generator(train_generator, epochs=3,
#                     validation_data=test_generator,
#                     shuffle=False,
#                     callbacks=[early_stopping])
history = model.fit(trainX, trainY, epochs=1, batch_size=16, validation_data = (testX,testY) , verbose=1,shuffle=False,
                    callbacks=[checkpoint])

1851/1851 [==============================] - ETA: 0s - loss: 0.0645 - mean_absolute_error: 0.1320
Epoch 1: loss improved from inf to 0.06448, saving model to y.hdf5
1851/1851 [==============================] - 118s 60ms/step - loss: 0.0645 - mean_absolute_error: 0.1320 - val_loss: 0.0021 - val_mean_absolute_error: 0.0407


In [94]:
predictions=model.predict_generator(testX)

In [95]:
print("predictions on test X")
print(predictions)

predictions on test X
[[-1.2781198]
 [-1.278151 ]
 [-1.2783172]
 ...
 [-1.2752383]
 [-1.2752142]
 [-1.2747202]]


In [96]:
prediction_copies = np.repeat(predictions, df_for_training.shape[1], axis=-1)
real_predictions = scaler.inverse_transform(prediction_copies)[:,-1]

In [97]:
print('predictions after inverse transforming')
print(real_predictions)

predictions after inverse transforming
[489.12555 489.03204 488.53473 ... 497.75006 497.82208 499.3006 ]


In [98]:
print("these are real values")
prediction_copies = np.repeat(testY, df_for_training.shape[1], axis=-1)
y = scaler.inverse_transform(prediction_copies)[:,-1]
print(y)

these are real values
[529. 480. 530. ... 327. 328. 292.]


In [99]:
rmse = np.sqrt(np.mean(((real_predictions - y) ** 2)))
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 137.84241019373107


In [100]:
import pickle
pickle.dump(model, open('model.pkl', 'wb'))

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......dense
.........vars
............0
............1
......dropout
.........vars
......dropout_1
.........vars
......leaky_re_lu
.........vars
......leaky_re_lu_1
.........vars
......lstm
.........cell
............vars
...............0
...............1
...............2
.........vars
......lstm_1
.........cell
............vars
...............0
...............1
...............2
.........vars
......lstm_2
.........cell
............vars
...............0
...............1
...............2
.........vars
...metrics
......mean
.........vars
............0
............1
......mean_absolute_error
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........17
.........18
.........19
.........2
.........20
.........21
.........22
.........3
.........4
.........5
.........6
.........7
.........8
.........9